In [1462]:
%matplotlib inline

In [1463]:
import sympy
import numpy as np
import matplotlib.pyplot as plt

from sympy.abc import x

Here are just some helper functions to be used in the ($k$, $n$)-threshold scheme. 

$prime\_generator$ gives us the prime number to be used throughout the encryption/decryption. We know that the prime number $p$ we need to use in Shamir's secret sharing must satisfy $p>max\{D, n\}$ where $D$ is the original data we wish to split into $n$ keys. (Not implemented)

$random\_distinct$ ensures that we get $n$ distinct random values $x_1,\ldots,x_n$. This is important because these random numbers will be used as input to the generated polynomial, so any $x_i=x_j$ will generate the exact same key and less than $n$ keys will be made.

$sample$ gives us a random sample of a list that we give it without replacement. Used to test the decryption with arbitrary k keys was created because numpy's $choice$ function will not work in our case.

In [1464]:
def prime_generator(lo, hi):
    ''' Generates a random prime number
    
        lo: Low bound for searching
        hi: High bound for searching
        
        Returns: Random prime within bounds
    '''
    
    return 1237 # totally random

def random_distinct(lo, hi, size):
    ''' Used to get lists of distinct random numbers,
    
        lo: Low bound for searching
        hi: High bound for searching
        
        Returns: Random numbers within bounds
    '''
    
    v = []
    
    while len(v) < size:
        rand = np.random.randint(lo, hi)
        
        if rand not in v:
            v.append(rand)
            
    return v

def sample(xs, size):
    copy = [ x for x in xs ]
    ys = []
    
    for i in range(size):
        idx = np.random.randint(0, len(copy)-1)
        ys.append(copy.pop(idx))
        
    return ys

Here you can change the values of $n$ and $k$ to see how it affects the program.

In [1465]:
# number of keys to generate
n = 50

# number of keys required to decrypt
k = 3

Here we will create the $(k-1)$-degree polynomial that will be used to generate the $n$ keys. It has the form $q(x)=a_{k-1}x^{k-1}+\ldots+a_2x^2+a_1x+a_0$, where $a_0=D$ and the $a_i$ are random integers.

In [1466]:
def generate_polynomial(data, k):
    ''' Makes a random polynomial
    
        data: The data as a numeric value to be split
        degree: The degree of the polynomial, and the minimum number of keys needed to decrypt
        
        Returns: polynomial as a function
    '''
    
    # will eventually give random primes greater than max{n, data}
    p = prime_generator(None, None)
    
    # Shamir's algorithms requires each coefficient to be distinct
    coefficients = random_distinct(0, p, k-1)
    
    # start building our polynomial
    polynomial = ''
    power = k-1
    
    while power > 0:
        # each a_i*x^i for i from k-1 to 1
        polynomial = polynomial + '{}*x**{}+'.format(coefficients[-power], power)
        power -= 1
        
    # set a_0 to data
    polynomial = polynomial + '{}'.format(data)
    
    # convert polynomial string to lambda function
    f = sympy.utilities.lambdify(x, sympy.sympify(polynomial))
    
    return polynomial, f, p


The output of the function is the polynomial represented as a string, a lambda function representing the polynomial, and the generated prime number to be used throughout. Shamir's paper does not specify whether or not the prime number is to be kept public or private, so more research is currently required on that detail.

In [1467]:
poly_str, poly_fn, prime = generate_polynomial(data=10, k=k)
poly_str

'1172*x**2+1008*x**1+10'

Here we generate the $n$ keys $D_1,\ldots,D_n$ evaluated as $q(x_i)=D_i$.

In [1468]:
def generate_keys(n, poly, p):
    ''' Creates the keys
    
        n: Number of keys to be created
        poly: Polynomial function to create the keys
        p: Prime number for modular arithmetic
        
        Returns: n-tuple of keys
    '''
    
    # create distinct x values because f(a)=f(b) iff a=b 
    # implies less than n keys will be made
    #X = random_distinct(1, p, n)
    X = range(1, n+1)
    
    # get corresponding y values
    Y = [poly(x) % p for x in X]
    
    return list(zip(X, Y))

keys = generate_keys(n=n, poly=poly_fn, p=prime)
keys

[(1, 953),
 (2, 529),
 (3, 1212),
 (4, 528),
 (5, 951),
 (6, 7),
 (7, 170),
 (8, 203),
 (9, 106),
 (10, 1116),
 (11, 759),
 (12, 272),
 (13, 892),
 (14, 145),
 (15, 505),
 (16, 735),
 (17, 835),
 (18, 805),
 (19, 645),
 (20, 355),
 (21, 1172),
 (22, 622),
 (23, 1179),
 (24, 369),
 (25, 666),
 (26, 833),
 (27, 870),
 (28, 777),
 (29, 554),
 (30, 201),
 (31, 955),
 (32, 342),
 (33, 836),
 (34, 1200),
 (35, 197),
 (36, 301),
 (37, 275),
 (38, 119),
 (39, 1070),
 (40, 654),
 (41, 108),
 (42, 669),
 (43, 1100),
 (44, 164),
 (45, 335),
 (46, 376),
 (47, 287),
 (48, 68),
 (49, 956),
 (50, 477)]

The way the decryption works is by reconstructing the original randomly created polynomial using $k$ keys. Each key has the form $(x_i, D_i)$. Using these $k$ points we can construct a Lagrange interpolating polynomial:

$$P(x)=\sum_{i=1}^kP_i(x)$$

$$P_i(x)=D_i\prod_{j\neq i}^k\frac{x-x_i}{x_j-x_i}$$

We can optimize it a bit since we only need to evaluate the polynomial at $x=0$ to get $a_0=D$. This will give us

$$P(0)=\sum_{i=1}^k\left(D_i\prod_{j\neq i}^k\frac{x_i}{x_i-x_j}\right)$$

In [1469]:
def decrypt(keys, p):
    ''' Uses Lagrange interpolation to decrypt data
    
        keys: List of keys to use in the decryption
        p: Prime number used to create finite field
        
        Returns: Decrypted data
    '''
    
    x, y = zip(*keys)
    summ = 0
    
    for i in range(len(y)):   
        product = y[i]
        
        for j in range(len(x)):      
            if i == j:
                continue
                
            product *= x[j]/(x[j]-x[i])

        summ += product
        
    return summ % p

def decrypt2(keys, p):
    a, b = zip(*keys)
    s = ''
    
    for i in range(len(b)):
        s = s + '{}'.format(b[i])
        
        for j in range(len(a)):
            if i == j:
                continue
                
            s = s + '*(x-{0})/({1}-{0})'.format(a[j], a[i])
        
        if i != len(b) - 1:
            s = s + '+'
           
    
    poly = sympy.sympify(s).simplify()
    print(poly)
    evald = poly.subs(x, 0)
    print(evald)
    num, den = sympy.fraction(evald)
    print(num, den)
    num = num % p
    den = den % p
    print(num, den)
    
    return num/den

Shamir's scheme requires $k$ keys in order to correctly recreate the original polynomial. The polynomial created with the $k$ points is unique, so any polynomial created with less than $k$ will be completely different, offering no information. However, our understanding of the Chinese Remainder Theorem is left wanting.

In [1523]:
k_keys = sample(keys, k)
k_keys

[(38, 119), (29, 554), (9, 106)]

In [1524]:
decrypt2(k_keys, prime)

-1061*x**2/435 + 50062*x/435 - 3661/5
-3661/5
-3661 5
50 5


10